In [258]:
# У меня почему-то не работал rnnmorph при стандартных настройках. 
# Пришлось понизить версии np и tf (гуглил)

# !pip3 install numpy==1.19.2
# !pip3 install tensorflow==2.4.0

In [2]:
import pandas as pd
import re
import numpy as np
import regex

In [3]:
from orv_cleanup_utils import *
from add_lib import absolute_lib_path

In [4]:
# путь к распарсенным отчётам на Ubuntu
data_dir = '/home/common/regulation_data/parsed_reports/'

# main_df - таблица, куда стожены заполнения one-to-one
main_df = pd.read_csv(data_dir + 'main_df.csv')

# загрузка one-to-many таблиц
otm_tables_fnames = sorted([fn for fn in os.listdir(data_dir) if not fn.startswith('main')])
otm_tables = {fn[:-4]: pd.read_csv(data_dir + fn) for fn in otm_tables_fnames}
otm_tables.keys()

dict_keys(['0_business', '1_business', '2_business', 'cancel_duties', 'expenses', 'goals', 'group_changes', 'group_expenses', 'groups', 'kpi', 'neccessary_measures', 'new_functions', 'notification_info', 'public_discussion', 'risks'])

In [5]:
# main_df.columns

Index(['bd_date', 'bd_impact_degree', 'developer', 'bd_year', 'path', 'type',
       'project_id', 'header: id', 'header: start', 'header: end',
       'general_info: Федеральный орган исполнительной власти (далее – разработчик)',
       'general_info: Сведения о федеральных органах исполнительной власти – соисполнителях',
       'general_info: Вид и наименование проекта акта',
       'general_info: Краткое описание проблемы, на решение которой направлен предлагаемый способ регулирования',
       'general_info: Основание для разработки проекта акта',
       'general_info: Краткое описание целей предлагаемого регулирования',
       'general_info: Краткое описание предлагаемого способа регулирования',
       'general_info: Контактная информация исполнителя разработчика',
       'degree: Степень регулирующего воздействия проекта акта',
       'degree: Обоснование отнесения проекта акта к определенной степени регулирующего воздействия',
       'problem: Описание проблемы, на решение которо

## Гайдлайны

Хорошее заполнение: ссылки на конкретные работы 

Отсутствие информации, непроведение оценки, указание закона, документа/ссылки.

In [6]:
def cleanup(text):
    text = str(text)
    text = text.lower()
    text = re.sub('[,\.]$', '', text)
    return text


def contains(text, pattern_list):
    for pattern in pattern_list:
        if pattern in text:
            return True

In [245]:
from collections import Counter


sources = main_df['groups: Источники данных'].apply(cleanup).to_list()
src_counter = Counter(sources)

### Обработка ссылок:

In [259]:
urls = [x for x in src_counter.keys() if contains(x, ['http', '.ru'])]
no_urls = {k: v for k, v in src_counter.items() if k not in urls}
len(urls)

139

In [248]:
# sorted(no_urls.items(), key=lambda x: (-x[1], x[0]))

### Пропуски:
- nan 
- \- 
- \---

In [249]:
def is_blank(text):
    if text in ['-', '_', 'nan']:
        return True
    text = text.replace('-', '').replace('_', '').replace('nan', '')
    if not text:
        return True
    
blanks = [x for x in no_urls.keys() if is_blank(x)]
no_blanks = {k: v for k, v in no_urls.items() if k not in blanks}
len(no_blanks)

1877

In [252]:
# sorted(no_blanks.items(), key=lambda x: (-x[1], x[0]))

[('отсутствуют', 1233),
 ('нет', 524),
 ('отсутствует', 434),
 ('минстрой россии', 166),
 ('информация отсутствует', 131),
 ('информация уточняется', 104),
 ('минпромторг россии', 86),
 ('росрыболовство', 83),
 ('информация росрыболовства', 74),
 ('самостоятельные исследования', 67),
 ('налоговый кодекс российской федерации', 60),
 ('информация о точном количестве участников отношений отсутствует', 58),
 ('законодательство российской федерации', 49),
 ('нет данных', 45),
 ('данные о субъектном составе группы участников отношений сформулированы исходя из существующей классификации участников перевозочного процесса, установленной законодательством российской федерации в области железнодорожного транспорта, а также исходя из существующей системы взаимоотношений между ними',
  43),
 ('открытые источники', 38),
 ('открытые данные', 35),
 ('росалкогольрегулирование', 32),
 ('статистическая отчетность ростехнадзора', 30),
 ('анализ рынка средств и услуг связи', 29),
 ('аналитическая информаци

### Отсутствие информации:

- (информация) 'отсутствует' во всех словоформах
- нет
- не имеется

In [253]:
import json
import requests


def use_yandexspeller(bad_str):
    """исправление орфографических ошибок с помощью Яндекс.Спеллер"""
    post_str = re.sub(r'[^\w|\d]', '', bad_str.lower())
    if not post_str:
        return None
    
    with requests.Session() as s:
        url = 'https://speller.yandex.net/services/spellservice.json/checkText?text='
        r = s.get(url + post_str)
        
        if r.status_code != 200:
            print(f'GET error: {r.status_code}')
            return None
        
        jsn = r.content.decode('unicode-escape')
        jsn_dict = json.loads(jsn)
        
        if not jsn_dict:
            return bad_str
        
        ya_answer = jsn_dict[0]['s']
        if ya_answer:
            return ya_answer[0]
        else:
            return bad_str

In [254]:
from rnnmorph.predictor import RNNMorphPredictor

def tokenize(sentence):
    sentence = re.sub('\(', ',', sentence)
    sentence = re.sub('\)', '', sentence)
    sentence = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', sentence)
    words = sentence.split()
    return words


def tag(sentence):
    words = tokenize(sentence)
    tagged_words = predictor.predict(words)
    return tagged_words


def preparse(sentence):
    tagged_words = tag(sentence)
    
    degree_re = re.compile('(.*)\|Degree=\w+(.*)')
    gender_re = re.compile('(.*)\|Gender=\w+(.*)')
    res = ''
    
    for word in tagged_words:
        word_tag = word.tag
        if 'Degree' in word_tag:
            word_tag = degree_re.sub(r"\1\2", word_tag)

        word_pos = 'pos=' + word.pos#.lower().capitalize()
        if 'PUNCT' in word_pos:
            word_pos = ''

        if 'ADJ' in word_pos:
            if 'Gender' in word_tag:
                word_tag = gender_re.sub(r"\1\2", word_tag)

        word_tags = [
                word.normal_form,
                word_pos,
                word_tag.replace('|', ' ').replace('_', ''),
            ]
        res += ' ' + ' '.join([tag for tag in word_tags if tag])

    res = res.strip().replace('  ', ' ')
    return res

def norm(sentence):
    if sentence is None:
        return
    tagged_words = tag(sentence)
    return ' '.join(word.normal_form for word in tagged_words)

In [255]:
predictor = RNNMorphPredictor(language="ru")

print (preparse('экспертная оценка сотрудников минтранса россии и росавиации'))
norm('экспертная оценка сотрудников минтранса россии и росавиации')

экспертный pos=ADJ Case=Nom Number=Sing оценка pos=NOUN Case=Nom Gender=Fem Number=Sing сотрудник pos=NOUN Case=Gen Gender=Masc Number=Plur минтранс pos=NOUN Case=Gen Gender=Masc Number=Sing россия pos=NOUN Case=Gen Gender=Fem Number=Sing и pos=CONJ росавиация pos=NOUN Case=Gen Gender=Fem Number=Sing


'экспертный оценка сотрудник минтранс россия и росавиация'

In [256]:
sorted(no_blanks.keys())

['(описание иной группы участников отношений n)',
 '*форма № 6-оип «сведения об использовании лесных участков, предоставленных в аренду, постоянное (бессрочное) и безвозмездное пользование за 2 полугодие 2019 г.», утвержденная приказом минприроды россии от 28.12.2015 № 565. **информация представлена рослесхозом',
 '- данные государственного охотхозяйственного реестра; - фонд данных государственного экологического мониторинга - данные росстата',
 '- информационные ресурсы росфинмониторинга; - официальный сайт фнс россии в информационно-телекоммуникационной сети «интернет»',
 '- план мероприятий («дорожной карты») по реализации механизма «регуляторной гильотины», утвержденный правительством российской федерации 29 мая 2019 г. № 4714п-п36;- федеральный закон № 244-фз; - положение',
 '- по оценке росстата на 1 января 2017 года - едина автоматизированная информационная система',
 '- реестр лицензий на осуществление деятельности по организации и проведению азартных игр в букмекерских контора

In [257]:
normed = {norm(str(use_yandexspeller(k))): v for k, v in no_blanks.items()}

In [284]:
no_info = ['нет', 'нет информация', 'не иметься', 'не применимый', 'не предусмотренный']
no_info_list = [x for x in normed.keys() if contains(x, ['отсутствовать']) or x in no_info]
has_info = {k: v for k, v in normed.items() if k not in no_info_list}

### Информация есть

In [285]:
# sorted(has_info.items(), key=lambda x: (-x[1], x[0]))

In [286]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/s.zorin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/s.zorin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [287]:
text = '\n'.join(src_counter.keys())
tokens = nltk.word_tokenize(text)
nltk_text = nltk.Text(tokens)

#### Реестры

In [288]:
nltk_text.similar('реестр')
# nltk_text.concordance('реестр')

перечень реестры информация федеральных регистр сайт статья аукционах
регистрация россвязи целом характеристика


In [289]:
registers = ['реестр', 'перечень', 'регистр']
registers_list = [x for x in has_info.keys() if contains(x, registers)]
src_counter_left = {k: v for k, v in has_info.items() if k not in registers_list}

In [290]:
print(len(src_counter_left.items()))

1491


In [291]:
# sorted(src_counter_left.items(), key=lambda x: (-x[1], x[0]))

#### Статистика

In [292]:
# for _ in statistics:
#     print(nltk_text.similar(_))
nltk_text.similar('данные')
# nltk_text.concordance('данные')

сведения информация и http проект отчеты от росстата органов положение
приказ переписка агентство перечень начиная заявление работа


In [293]:
nltk_text.similar('данные')

сведения информация и http проект отчеты от росстата органов положение
приказ переписка агентство перечень начиная заявление работа


In [294]:
statistics = ['данные', 'отчет', 'отчётность', 'анализ', 'сведения',
              'исследование', 'информация', 'доклад', 'статистика']
statistics_list = {x for x in has_info.keys() if contains(x, statistics)}
src_counter_left = {k: v for k, v in has_info.items() if k not in statistics_list}

In [295]:
print(len(src_counter_left.items()))

1178


In [296]:
# sorted(src_counter_left.items(), key=lambda x: (-x[1], x[0]))

#### Законодетальство

In [297]:
laws = ['федеральный закон', 'кодекс', 'постановление' 'конституция'] 
laws_list = {x for x in has_info.keys() if contains(x, laws)}
src_counter_left = {k: v for k, v in has_info.items() if k not in laws_list}

In [298]:
print(len(src_counter_left.items()))

1460


In [299]:
d = registers_list
d.extend(statistics_list)
d.extend(laws_list)
len(d)

1235

In [300]:
# sorted(src_counter_left.items(), key=lambda x: (-x[1], x[0]))

In [302]:
left = {k: v for k, v in has_info.items() if not contains(x, registers_list) and not contains(x, statistics_list) and containts(x, laws_list)}
sorted(left.items(), key=lambda x: (-x[1], x[0]))

NameError: name 'x' is not defined